<a href="https://colab.research.google.com/github/xcellentbird/Machine_Learning/blob/main/3%EB%B6%84_torch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 50
BATCH_SIZE = 128
'can i use cuda?',torch.cuda.is_available()

('can i use cuda?', True)

In [5]:
transform = transforms.Compose([
                                transforms.ToTensor(),
                                # grayscale일 때, normalize에 mean, std는 1개의 값이 들어간다.
                                # colorscale일 때는 3개의 값
                                transforms.Normalize(mean=(0.1307,), std=(0.3081,))
])

In [8]:
train_dataset = datasets.MNIST('./', train = True, download = True, transform = transform)
test_dataset = datasets.MNIST('./', train = False, download = True, transform = transform)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [9]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [24]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10)

  def forward(self, x):
    # input_size = 28 * 28 * 1
    x = self.conv1(x) # ouput_size = input - kernel_size//2 * 2 = 28 - 5//2 * 2 = 24(24 * 24 * 10)
    x = F.max_pool2d(x, 2) # output_size = input / pool_size = 24 / 2 = 12(12 * 12 * 10)
    x = F.relu(x)
    x = self.conv2(x) # output_size = 12 - 5//2 * 2 = 8(8 * 8 * 20)
    x = self.conv2_drop(x)
    x = F.max_pool2d(x, 2) # output_size = 8 / 2 = 4(4 * 4 * 20 = 320)
    x = F.relu(x)
    x = x.view(-1, 320)
    x = self.fc1(x)
    x = F.relu(x)
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    x = F.log_softmax(x, dim=1)
    return x


In [25]:
model = CNN().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [36]:
def train(model, train_loader, optimizer, epoch):
  model.train()
  for idx, (data, target) in enumerate(train_loader):
    data, target = data.to(DEVICE), target.to(DEVICE)
    optimizer.zero_grad()
    output = model(data)
    
    loss = F.cross_entropy(output, target)
    loss.backward()
    optimizer.step()

    if not idx % 200:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, idx * len(data), len(train_loader.dataset),
          100. * idx / len(train_loader), loss.item()
      ))

In [39]:
def evaluate(model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(DEVICE), target.to(DEVICE)
      output = model(data)

      test_loss += F.cross_entropy(output, target, reduction='sum').item()

      _, pred = output.max(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()
  test_accuracy = 100. * correct / len(test_loader.dataset)
  return test_loss, test_accuracy

In [40]:
for epoch in range(EPOCHS):
  train(model, train_loader, optimizer, epoch+1)
  test_loss, test_accuracy = evaluate(model, test_loader)

  print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
      epoch+1, test_loss, test_accuracy
  ))

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.144554
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.108583
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.127664
[1] Test Loss: 330.5007, Accuracy: 99.02%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.086911
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.056149
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.103242
[2] Test Loss: 319.3181, Accuracy: 98.97%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.118272
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.132753
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.081082
[3] Test Loss: 342.3056, Accuracy: 98.84%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.136692
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.108292
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.128768
[4] Test Loss: 344.6239, Accuracy: 98.87%
Train Epoch: 5 [0/60000 (0%)]	Loss: 0.138449
Train Epoch: 5 [25600/60000 (43%)]	Loss: 0.075355
Train Epoch: 5 [51200/60000 (85%)]	Loss: 0.155837
[5] Test Loss: 343.1648, Accuracy: 99.00%
Train Epoch: 6 [0/60000 (0%)]	Loss: 0.047920
Train Epoch: 6 [2560